# New Section

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np
import os
from google.colab import files
import io # Pour lire le fichier uploadé

print("Libraries installed and imported successfully!")

Libraries installed and imported successfully!


In [5]:
try:
    df = pd.read_csv('/content/Plan-comptable-Maroc-Excel.csv')
    print(f"\nSuccessfully loaded DataFrame with {df.shape[0]} rows and {df.shape[1]} columns.")
    print("\nFirst 5 rows of the data:")
    display(df.head()) # display() is nicer in Colab


    # Basic Data Cleaning / Preparation
    # Ensure required columns exist
    required_cols = ['N° de compte', 'Nomenclature du compte']
    if not all(col in df.columns for col in required_cols):
        print(f"\nError: Missing one or more required columns: {required_cols}")
        raise ValueError("DataFrame missing required columns.")
    # *** NEW CLEANING STEP: Remove rows where 'N° de compte' length is 1, 2, or 3 ***
    print("\nFiltering accounts based on length...")
    rows_before_filter = df.shape[0]
    # Keep rows where the length of the 'N° de compte' string is greater than 3
    df = df[df['N° de compte'].str.len() > 3]
    rows_after_filter = df.shape[0]
    print(f" - Removed {rows_before_filter - rows_after_filter} rows where 'N° de compte' length was 1, 2, or 3.")
    print(f" - Kept {rows_after_filter} rows.")
    # *** END OF NEW CLEANING STEP ***
    # Remove rows where 'Nomenclature du compte' is missing (if any)
    df.dropna(subset=['Nomenclature du compte'], inplace=True)
    # Convert account number to string to handle cases like '24 - 25' or potential leading zeros if needed later
    df['N° de compte'] = df['N° de compte'].astype(str)
    # Convert nomenclature to string
    df['Nomenclature du compte'] = df['Nomenclature du compte'].astype(str)
    print(f"\nData prepared. Using {df.shape[0]} account entries.")
except Exception as e:
    print(f"\nAn error occurred while loading or processing the CSV: {e}")
    raise


Successfully loaded DataFrame with 1239 rows and 4 columns.

First 5 rows of the data:


,N° de compte,Nomenclature du compte,La classe,La rubrique
0,1,Comptes de Financement Permanent,Financement Permanent,NaN
1,11,Capitaux Propres,Comptes de Financement Permanent,Capitaux propres
2,1111,Capital Social,Comptes de Financement Permanent,Capitaux propres
3,1112,Fonds de Dotation,Comptes de Financement Permanent,Capitaux propres
4,1117,Capital Personnel,Comptes de Financement Permanent,Capitaux propres



Filtering accounts based on length...
 - Removed 239 rows where 'N° de compte' length was 1, 2, or 3.
 - Kept 1000 rows.

Data prepared. Using 1000 account entries.


In [6]:
display(df.head())

,N° de compte,Nomenclature du compte,La classe,La rubrique
2,1111,Capital Social,Comptes de Financement Permanent,Capitaux propres
3,1112,Fonds de Dotation,Comptes de Financement Permanent,Capitaux propres
4,1117,Capital Personnel,Comptes de Financement Permanent,Capitaux propres
5,11171,Capital Individuel,Comptes de Financement Permanent,Capitaux propres
6,11175,Compte de l'Exploitant,Comptes de Financement Permanent,Capitaux propres


In [7]:
model_name = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
print(f"Loading sentence transformer model: {model_name}...")
# Check if GPU is available and use it
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
model = SentenceTransformer(model_name, device=device)
print("Model loaded successfully.")

Loading sentence transformer model: sentence-transformers/paraphrase-multilingual-mpnet-base-v2...
Using device: cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model loaded successfully.


In [8]:
account_descriptions = df['Nomenclature du compte'].tolist()

print(f"Generating embeddings for {len(account_descriptions)} account descriptions...")

# Generate embeddings. Use show_progress_bar=True for visual feedback.
# Using batching internally for efficiency.
account_embeddings = model.encode(account_descriptions, convert_to_tensor=True, show_progress_bar=True)

print(f"Embeddings generated successfully. Shape: {account_embeddings.shape}")

Generating embeddings for 1000 account descriptions...


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Embeddings generated successfully. Shape: torch.Size([1000, 768])


In [9]:
def find_matching_account(query, df_accounts, embeddings, model, threshold=0.70, top_n=1):
    results = []

    # 1. Check for Exact Match (Case-Insensitive) first for perfect precision
    df_accounts['Nomenclature Lower'] = df_accounts['Nomenclature du compte'].str.lower()
    query_lower = query.lower()
    exact_match = df_accounts[df_accounts['Nomenclature Lower'] == query_lower]

    if not exact_match.empty:
        print("--- Found Exact Match ---")
        for index, row in exact_match.iterrows():
             results.append({
                'N° de compte': row['N° de compte'],
                'Nomenclature': row['Nomenclature du compte'],
                'Score': 1.0 # Perfect score for exact match
            })
        df_accounts.drop(columns=['Nomenclature Lower'], inplace=True) # Clean up temp column
        return results[:top_n] # Return only the top N (usually 1 for exact)

    # Cleanup the temporary column if no exact match found
    df_accounts.drop(columns=['Nomenclature Lower'], inplace=True)


    # 2. If no exact match, proceed with Semantic Search
    print("--- Performing Semantic Search ---")
    # Encode the query
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Compute cosine similarity between the query and all account embeddings
    # util.cos_sim returns a tensor of shape [1, num_accounts]
    cosine_scores = util.cos_sim(query_embedding, embeddings)[0] # Get the scores for the single query

    # Find the top N matches above the threshold
    # Use torch.topk to get indices and scores of the best matches
    top_results = torch.topk(cosine_scores, k=top_n)

    print(f"Top {top_n} raw scores: {top_results.values.tolist()}") # Show raw scores

    for i in range(top_n):
        score = top_results.values[i].item() # .item() converts tensor scalar to Python number
        idx = top_results.indices[i].item()

        if score >= threshold:
            match_info = {
                'N° de compte': df_accounts.iloc[idx]['N° de compte'],
                'Nomenclature': df_accounts.iloc[idx]['Nomenclature du compte'],
                'Score': round(score, 4) # Round for readability
            }
            results.append(match_info)
        else:
            # If the top score is already below threshold, others will be too
            print(f"Top match score ({round(score, 4)}) is below threshold ({threshold}).")
            break # Stop adding results if score drops below threshold

    if not results:
         print("No sufficiently similar match found above the threshold.")

    return results

In [10]:
queries = [
    "Capital Social", # Should be an exact match
    "Amortissement matériel informatique", # Should find 28355
    "frais de démarrage", # Should find 2112
    "TVA récupérable sur charges", # Should find 34552
    "achats marchandises groupe a", # Should find 6111
    "compte de l'exploitant", # Should find 11175
    "dettes fournisseurs", # Should find 4411 (or related)
    "argent en banque", # Ambiguous, let's see what it finds
    "quelque chose de très vague", # Likely below threshold
    "Résultat net de l'exercice (Solde créditeur)" # Exact match
]

# --- Run Tests ---
for q in queries:
    print(f"\n\n==================\nQuery: '{q}'\n==================")
    matches = find_matching_account(q, df, account_embeddings, model, threshold=0.75, top_n=3) # Using threshold 0.75, get top 3
    if matches:
        for match in matches:
            print(f"  -> N° Compte: {match['N° de compte']}")
            print(f"     Nom:       {match['Nomenclature']}")
            print(f"     Score:     {match['Score']:.4f}")
    else:
        print("  -> No suitable match found.")



Query: 'Capital Social'
--- Found Exact Match ---
  -> N° Compte: 1111
     Nom:       Capital Social
     Score:     1.0000


Query: 'Amortissement matériel informatique'
--- Performing Semantic Search ---
Top 3 raw scores: [0.9853929877281189, 0.8156975507736206, 0.8048076033592224]
  -> N° Compte: 28355
     Nom:       Amortissement du matériel informatique
     Score:     0.9854
  -> N° Compte: 61315
     Nom:       Location de matériel informatique
     Score:     0.8157
  -> N° Compte: 28352
     Nom:       Amortissement du matériel de bureau
     Score:     0.8048


Query: 'frais de démarrage'
--- Performing Semantic Search ---
Top 3 raw scores: [0.9227770566940308, 0.8771371841430664, 0.8611372709274292]
  -> N° Compte: 28112
     Nom:       Amortissement des frais préliminaires au démarrage
     Score:     0.9228
  -> N° Compte: 2118
     Nom:       Autres frais préliminaires
     Score:     0.8771
  -> N° Compte: 2811
     Nom:       Amortissements des frais préliminaires
 